# ElGamal

## Domain Parameter

In [2]:
p = 59
g = 4 # ord(g) = 29

## Key Generation

In [3]:
# Alice send public key to Bob
a = 6
A = pow(g, a, p)
pub = (p, g, A)

## Encryption

In [4]:
p, g, A = pub
x = 18
# Ephemeral key
import random
i = random.randint(2,p-2)
kE = pow(g, i, p)
# Masking key
kM = pow(A, i, p)
y = (x*kM) % p
cipher = (kE, y)
print (kM)

27


## Decryption

In [6]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, p):
    g, x, y = egcd(a, p)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % p

kE, y = cipher
kM = pow(kE, a, p)
kMinv = modinv(kM, p)
x = (y * kMinv) % p

print (x)

18


## Problems

### Get Primitive Elements of 59

In [7]:
for i in range(1, 59):
    if pow(i, 2, 59) != 1 and pow(i,29, 59) != 1 and pow(i,58, 59) == 1:
        print (i)

2
6
8
10
11
13
14
18
23
24
30
31
32
33
34
37
38
39
40
42
43
44
47
50
52
54
55
56


### Textbook 8.18 - If ord(public_key) is small
- p=29, g=2
- public key A = 28

In [8]:
def dec(cipher, a):
    p = 29
    g = 2
    kE, y = cipher
    kM = pow(kE, a, p)
    kMinv = modinv(kM, p)
    x = (y * kMinv) % p
    return x

# log_g A mod p
def decrete_log(g, A):
    p = 29
    for i in range(2,p):
        if pow(g, i, 29) == A:
            return i
    return 0

ciphers = [(3,15),(19,14),(6,15),(1,24),(22,13),(4,7),(13,24),(3,21),(18,12),(26,5),(7,12)]
table = 'abcdefghijklmnopqrstuvwxyzäöü'

p = 29
g = 2

- All possible plaintext char

In [9]:
dec = ''
for c in ciphers:
    kE, y = c
    kM = 28
    kMinv = kM
    dec += table[(y*kMinv)%p]
print (dec)

dec = ''
for c in ciphers:
    kE, y = c
    kM = 1
    kMinv = kM
    dec += table[(y*kMinv)%p]
print (dec)

opofqwfiryr
popynhyvmfm


- Determine kM = 1 or kM = 28 by Quadratic Residuosity
    - $k_E$ is $g^i$$modP$
    - if $k_E$ is quadratic residual, then $i$ must be even, 
        then $k_M$ is 1; otherwise $k_M$ is 28

In [10]:
ciphers = [(3,15),(19,14),(6,15),(1,24),(22,13),(4,7),(13,24),(3,21),(18,12),(26,5),(7,12)]
table = 'abcdefghijklmnopqrstuvwxyzäöü'

p = 29
g = 2

dec = ''
residual_2_29 = []
for i in range(2,29,2):
    residual_2_29.append(pow(2, i, 29))
print (residual_2_29)
for c in ciphers:
    kE, y = c
    if kE in residual_2_29:
        kM = 1
    else:
        kM = 28
    kMinv = kM
    dec += table[(y*kMinv) % p]
print (dec)    

[4, 16, 6, 24, 9, 7, 28, 25, 13, 23, 5, 20, 22, 1]
oppynhyirym


- Cheat

In [11]:
# Compute i from Ephermeral key (cheat)
dec = ''
for c in ciphers:
    kE, y = c
    i = decrete_log(g, kE)
    kM = pow(28, i, p)
    kMinv = kM
    dec += table[(y*kMinv)%p]
print (dec)

# Compute Bob's private key (cheat)
dec = ''
priv = decrete_log(g, 28)
for c in ciphers:
    kE, y = c
    kM = pow(kE, priv, p)
    kMinv = modinv(kM, p)
    dec += table[(y*kMinv)%p]
print (dec)

oppynhyirym
oppynhyirym


### Domain Parameter for ElGamel (or DSA)
- For which prime numbers p and q, a multiplicative cyclic group of order q can be constructed as a subgroup of $(Z_p^*,\times)$ ?
- $q | (p-1)$

In [12]:
print (862 % 109)
print (856 % 107)
print (858 % 103)
print (852 % 101)

99
0
34
44


## Shank's Baby-Step Giant-Step
- Example:
Solve $5^x≡52(mod307)$
    - Write $x=i+18k$ where $0\leq i,k<18$ (18 is the ceiling of $\sqrt{307}$) 
        - That is $5^i≡52\times5^{-18k}(mod307)$
    - List baby steps $(i,5^i)$ and giant steps $(k,52\times5^{-18k})$
    - Find the collision

In [21]:
from math import sqrt, ceil
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, p):
    g, x, y = egcd(a, p)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % p
def babystep_giantstep(x,y,p):
    baby_steps = []
    sqrt_p = ceil(sqrt(p))
    # baby step
    for i in range(sqrt_p):
        baby_steps.append(pow(x, i, p))
        print (i, pow(x, i, p))

    # giant step
    giant = modinv(pow(x,sqrt_p,p), p) # in this example, 5^(-18)
    for k in range(sqrt_p):
        giant_step = y*pow(giant, k, p)%p
        print (k, y*pow(giant, k, p)%p)
        # check
        for i in range(sqrt_p):
            if giant_step in baby_steps:
                return baby_steps.index(giant_step), k

        
p = 307
x = 5
y = 52
print (babystep_giantstep(x,y,p))

0 1
1 5
2 25
3 125
4 11
5 55
6 275
7 147
8 121
9 298
10 262
11 82
12 103
13 208
14 119
15 288
16 212
17 139
0 52
1 247
2 22
3 258
4 151
5 180
6 241
7 147
(7, 7)
